In [ ]:
!pip install sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


sentences = [
    "Bu gün universitetdə süni intellekt mövzusunda seminar keçirildi.",
    "Yeni tədris platforması tələbələr üçün istifadəyə verildi.",
    "İnformasiya təhlükəsizliyi mövzusunda təlim təşkil olundu.",
    "Magistratura üzrə qəbul qaydaları yeniləndi.",
    "Tələbələrin startap layihələri üçün yeni fond yaradıldı.",
    "Universitet müəllimləri üçün rəqəmsal bacarıqlar təlimi keçirildi.",
    "Yeni araşdırma mərkəzində robototexnika laboratoriyası açıldı."
]


embeddings = model.encode(sentences, convert_to_tensor=True)

print("Dataset hazırdır, embedding-lər generasiya olundu.")


In [ ]:
query = "Universitetdə AI haqqında tədbir baş tutdu"
query_emb = model.encode(query, convert_to_tensor=True)

print("Sorğu embedding-i hazırdır.")


In [ ]:

scores = util.cos_sim(query_emb, embeddings)[0]


for idx, score in enumerate(scores):
    print(f"{idx}. {sentences[idx]}  ---> Oxşarlıq: {score:.4f}")


In [ ]:
import torch


best_idx = torch.argmax(scores).item()

print("\n📌 Ən uyğun cümlə:")
print(sentences[best_idx])
print(f"Oxşarlıq dərəcəsi: {scores[best_idx]:.4f}")


In [ ]:

sorted_results = torch.argsort(scores, descending=True)

print("📌 Ən uyğun 3 nəticə:")
for idx in sorted_results[:3]:
    print(f"- {sentences[idx]} (oxşarlıq: {scores[idx]:.4f})")


In [ ]:
#Sentiment

In [ ]:
#Sentiment

In [ ]:
#Sentiment

In [ ]:
!pip install transformers sentencepiece
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("Model yükləndi.")

In [ ]:
import numpy as np

labels = {
    0: "Çox mənfi ",
    1: "Mənfi ",
    2: "Neytral ",
    3: "Müsbət ",
    4: "Çox müsbət"
}

def sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=1)[0].detach().numpy()
    label_id = np.argmax(probabilities)
    return labels[label_id], float(probabilities[label_id])


In [ ]:
text = "Bu gün dərs çox maraqlı keçdi, özümü əla hiss edirəm!"
result, prob = sentiment(text)

print(f"Mətn: {text}")
print(f"Sentiment: {result} (ehtimal: {prob:.4f})")


In [ ]:
sentences = [
    "Universitetdə xidmət səviyyəsi çox zəif idi.",
    "Bu layihə məni çox həyəcanlandırır!",
    "Təlim normal keçdi.",
    "Sistemdə xeyli problemlər var idi, narazı qaldım.",
    "Yeni platforma gözlədiyimdən daha yaxşı işləyir."
]

for s in sentences:
    label, prob = sentiment(s)
    print(f"{s} ---> {label} (ehtimal: {prob:.4f})")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

probs = []
for s in sentences:
    inputs = tokenizer(s, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    p = torch.softmax(outputs.logits, dim=1)[0].detach().numpy()
    probs.append(p)

plt.figure(figsize=(10,4))
sns.heatmap(probs, annot=True, cmap="Blues",
            yticklabels=sentences,
            xticklabels=list(labels.values()))
plt.title("Azərbaycan cümlələri üzrə Sentiment heatmap")
plt.show()


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
def sentiment_grouped(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0].detach().numpy()


    stars = np.argmax(probs) + 1

    if stars in [1, 2]:
        label = "Mənfi"
    elif stars == 3:
        label = "Neytral"
    else:
        label = "Müsbət"

    return stars, label, probs
sentences = [
    "Universitetdə xidmət səviyyəsi çox zəif idi.",
    "Bu layihə məni çox həyəcanlandırır!",
    "Təlim normal keçdi.",
    "Sistemdə xeyli problemlər var idi, narazı qaldım.",
    "Yeni platforma gözlədiyimdən daha yaxşı işləyir."
]

for s in sentences:
    stars, label, probs = sentiment_grouped(s)
    print(f"{s}  →  {label} ({stars}★)")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ["Mənfi", "Neytral", "Müsbət"]

def sentiment_xlm(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0].detach().numpy()
    label_id = np.argmax(probs)
    return labels[label_id], probs[label_id], probs
sentences = [
    "Universitetdə xidmət səviyyəsi çox zəif idi.",
    "Bu layihə məni çox həyəcanlandırır!",
    "Təlim normal keçdi.",
    "Sistemdə xeyli problemlər var idi, narazı qaldım.",
    "Yeni platforma gözlədiyimdən daha yaxşı işləyir."
]

for s in sentences:
    label, prob, raw = sentiment_xlm(s)
    print(f"{s}  →  {label} (ehtimal: {prob:.3f})")
